# QPT consisting from the 	arXiv:2006.02424 article

In [4]:
using PastaQ
using Random
using ITensors
using Observers
using Optimisers: Optimisers
using Test
using LinearAlgebra
using SCS
using Convex

N = 2
nshots = 1
gates = randomcircuit(N; depth=10)
Λ1 = runcircuit(gates; process=true)

print("Run circuit MPO:")
display(Λ1)

#For training data
preps = randompreparations(N,4) #random initialization of states
bases = randombases(N,4)        #random basis of measurements
data = getsamples(Λ1, preps, bases,100) #Get samples for these datapoints accordingly to the preparation and basis

#For testing data
preps = randompreparations(N,4)
bases = randombases(N,4)
target = getsamples(Λ1, preps, bases, 2)

print("Part 1:----------------------------------")
#display(data)
print("-----------------------------------------")

χ = maxlinkdim(Λ1) #
U0 = randomprocess(Λ1,mixed=false; χ=χ) #definition of the variational circuit
print("This is U0:")
display(U0)
print("Part 2---------------------------------------------")

#F(bu::LPDO; kwargs...) = fidelity(bu, Λ; process=true)
#obs = Observer(["F" => F])

print("Part 3---------------------------------------------")

N = size(data, 2) # Number of qubits

opt = Optimisers.ADAM() #optimizer
#opt=Optimisers.Descent(0.01)

# Run quantum state tomography
Λ = tomography(data, U0; optimizer = opt, test_data = target,target=Λ1, batchsize=500,epochs=100) #observer=obs,
#display(Λ)

Run circuit MPO:

MPO
[1] ((dim=2|id=499|"Qubit,Site,n=1")', (dim=2|id=499|"Qubit,Site,n=1"), (dim=4|id=176|"Link,n=1"))
[2] ((dim=4|id=176|"Link,n=1"), (dim=2|id=448|"Qubit,Site,n=2")', (dim=2|id=448|"Qubit,Site,n=2"))


Part 1:---------------------------------------------------------------------------This is U0:

MPO
[1] ((dim=2|id=499|"Qubit,Site,n=1")', (dim=4|id=488|"Link,l=1"), (dim=2|id=499|"Qubit,Site,n=1"))
[2] ((dim=2|id=448|"Qubit,Site,n=2")', (dim=4|id=488|"Link,l=1"), (dim=2|id=448|"Qubit,Site,n=2"))


Part 2---------------------------------------------Part 3---------------------------------------------1     ⟨logP⟩ = 3.4220  (2.7769)  elapsed = 0.158s
2     ⟨logP⟩ = 3.2928  (2.8446)  elapsed = 0.148s
3     ⟨logP⟩ = 3.1704  (2.8470)  elapsed = 0.137s
4     ⟨logP⟩ = 3.0759  (2.8477)  elapsed = 0.138s
5     ⟨logP⟩ = 2.9947  (2.8010)  elapsed = 0.148s
6     ⟨logP⟩ = 2.9101  (2.7600)  elapsed = 0.141s
7     ⟨logP⟩ = 2.8450  (2.7550)  elapsed = 0.145s
8     ⟨logP⟩ = 2.7817  (2.8069)  elapsed = 0.149s
9     ⟨logP⟩ = 2.7317  (2.7818)  elapsed = 0.147s
10    ⟨logP⟩ = 2.6942  (2.7904)  elapsed = 0.133s
11    ⟨logP⟩ = 2.6662  (2.8219)  elapsed = 0.148s
12    ⟨logP⟩ = 2.6487  (2.8937)  elapsed = 0.151s
13    ⟨logP⟩ = 2.6276  (2.9672)  elapsed = 0.141s
14    ⟨logP⟩ = 2.6091  (2.8480)  elapsed = 0.145s
15    ⟨logP⟩ = 2.6015  (2.7915)  elapsed = 0.146s
16    ⟨logP⟩ = 2.5879  (2.7515)  elapsed = 0.152s
17    ⟨logP⟩ = 2.5832  (2.7105)  elapsed = 0.138s
18    ⟨logP⟩ = 2.5674  (2.6804)  elapsed = 0.141

MPO
[1] ((dim=2|id=499|"Qubit,Site,n=1")', (dim=4|id=488|"Link,l=1"), (dim=2|id=499|"Qubit,Site,n=1"))
[2] ((dim=2|id=448|"Qubit,Site,n=2")', (dim=4|id=488|"Link,l=1"), (dim=2|id=448|"Qubit,Site,n=2"))


In [5]:
#Λ1,Λ - quantum circuit tensor network and variational tensor network

#Variational vector format
Λmat = PastaQ.array(Λ)
Λvec = vec(Λmat)

print("Variational tensor network:")
print(" ")
display(Λvec)

#Random circuit vector format
Λmat1 = PastaQ.array(Λ1)
Λvec1 = vec(Λmat1)


print("Quantum circuit:")
print(" ")
display(Λvec1)

MSE=0
for i in 1:size(Λvec, 1)
    MSE=abs(Λvec[i]*Λvec[i]-Λvec1[i]*Λvec1[i]) + MSE
end 

#MSE=MSE/size(Λvec, 1)

print("MSE non normalized:")
display(MSE)

Variational tensor network: 

16-element Vector{ComplexF64}:
  0.32694454801842465 - 0.1548999401530655im
  0.07124964675895713 - 0.08673876713212328im
   0.2050340029207398 + 0.1636846887527228im
   0.2838072270609102 + 0.11967967849430342im
  -0.2871216776588283 + 0.1906848270720099im
  -0.1284285620990518 + 0.09408431670739281im
   0.2742135626761631 - 0.04704482408482255im
    0.417332269640178 + 0.038185920679627755im
 -0.25756649126832026 + 0.9006011067745788im
 -0.21632199247384862 + 0.15392591417074847im
  0.47979967416647473 + 0.12354013212773277im
  -0.3436374826754379 - 0.6147666368025322im
 -0.08213074248157418 - 0.3522964705002836im
  0.10674324524442917 - 0.34429432807522825im
  -0.9069311730367954 + 0.6885977589722938im
  0.17932408604987046 + 0.12880516021918306im

Quantum circuit: 

16-element Vector{ComplexF64}:
  -0.5499955021873278 + 0.465949923028582im
  -0.5877772297653241 - 0.0030805345936950335im
 -0.30762028875768577 + 0.06470907011427987im
 -0.16160845011106373 + 0.09984617329505466im
 -0.32422564218074956 - 0.4461071757106782im
  0.10254679981404347 + 0.29549040358646195im
  0.10175162088147957 + 0.6179860788000766im
 -0.29442029505836526 + 0.3450973773924977im
  0.11750572617652132 - 0.2373181483627418im
 -0.16011981594290955 - 0.06573922664512229im
  -0.3702641968236978 - 0.08116648536538436im
   0.5748917658148933 + 0.6524786457793661im
  -0.3112348684155348 - 0.09626575191139203im
    0.287230560863912 + 0.6664957743605957im
  -0.2605403523689795 - 0.5453049231744305im
 -0.02159105360966708 - 0.037984751016368834im

MSE non normalized:

6.109054442521786

# Aux codes

In [2]:
#using JLD2

#hello_loaded = load_object("dataB1.h5")
#display(hello_loaded)
#hello_loaded

┌ Warning: This file was not written with JLD2. Some things may not work.
└ @ JLD2 C:\Users\vladl\.julia\packages\JLD2\ryhNR\src\JLD2.jl:379


8×8 Matrix{NamedTuple{(:r, :i), Tuple{Float64, Float64}}}:
 (r = 0.0536476, i = 0.0)         …  (r = 0.0191324, i = 0.0351649)
 (r = 0.0288901, i = 0.100742)       (r = -0.0557315, i = 0.0548649)
 (r = 0.019322, i = -0.00733856)     (r = 0.0117011, i = 0.010048)
 (r = 0.0844858, i = 0.134043)       (r = -0.0577319, i = 0.103183)
 (r = 0.00107314, i = 0.0299312)     (r = -0.0192365, i = 0.0113778)
 (r = 0.0380839, i = 0.0178625)   …  (r = 0.00187343, i = 0.0313335)
 (r = 0.0301154, i = 0.095275)       (r = -0.0517107, i = 0.0537181)
 (r = 0.0191324, i = -0.0351649)     (r = 0.0298731, i = 0.0)

8×8 Matrix{NamedTuple{(:r, :i), Tuple{Float64, Float64}}}:
 (r = 0.0536476, i = 0.0)         …  (r = 0.0191324, i = 0.0351649)
 (r = 0.0288901, i = 0.100742)       (r = -0.0557315, i = 0.0548649)
 (r = 0.019322, i = -0.00733856)     (r = 0.0117011, i = 0.010048)
 (r = 0.0844858, i = 0.134043)       (r = -0.0577319, i = 0.103183)
 (r = 0.00107314, i = 0.0299312)     (r = -0.0192365, i = 0.0113778)
 (r = 0.0380839, i = 0.0178625)   …  (r = 0.00187343, i = 0.0313335)
 (r = 0.0301154, i = 0.095275)       (r = -0.0517107, i = 0.0537181)
 (r = 0.0191324, i = -0.0351649)     (r = 0.0298731, i = 0.0)

In [1]:

# Set the random seed so the results are the same
# each time it is run
#Random.seed!(1234)

# Initialize the MPS state ψ = |0,0,0⟩
#ψ = qubits(3)
#N=3
#nbases=3
# Apply the X gate on qubit 2
#g = ("H", 2)
#ψ = runcircuit(ψ, g)
#bases= randombases(N, nbases)
#A=getsamples(ψ,bases)
#display(A)

# Show samples from P(x) = |⟨H|ψ⟩|²
#println("Sample from |ψ⟩ = H2|0,0,0⟩:")
#display(getsamples(ψ, 50))
#println()

In [4]:
#"Small amount of code to understand page 14 of article"

"Small amount of code to understand page 14 of article"

In [5]:

#@testset "Trace preserving condition in QPT" begin
#  Random.seed!(1234)
#  N = 2
#  d = 1 << N
#  nshots = 3
#  gates = randomcircuit(N; depth=4)
#  Λ = runcircuit(gates; process=true, noise=("DEP", (p=0.001,)))
#  preps = fullpreparations(N)
#  bases = fullbases(N)
#  data = getsamples(Λ, preps, bases, nshots)

#  ρ = tomography(data; method="LS")
#  for j in 1:N
#    s = firstind(ρ; tags="Output,n=$(j)", plev=0)
#    ρ = ρ * δ(s, s')
#  end

#  ρmat = PastaQ.array(ρ)
#  @test ρmat ≈ Matrix{Float64}(I, d, d) atol = 1e-5
#  print(ρmat)
#end


ComplexF64[0.9999999962212077 - 7.132082919106261e-19im 3.0797378536284725e-12 + 1.4025551553498161e-12im -2.7143010061791983e-12 + 8.54011306117286e-13im 4.598169067726587e-12 - 1.588958999099166e-12im; 1.8848880789512634e-12 - 1.4026280137358071e-12im 0.9999999962178888 - 1.7384894854275448e-19im -4.5314446639466155e-13 - 3.5324937419645153e-12im 2.3620203015717323e-12 - 2.743430482787801e-12im; -1.6474044350900385e-12 - 8.539835505416704e-13im -2.6251917306652217e-13 + 3.532424353025476e-12im 0.9999999962144832 + 2.398336817571067e-19im -1.6988563333875106e-12 + 1.0667786098927934e-12im; 2.7987473449897493e-12 + 1.5890405311025368e-12im 1.440033856048295e-12 + 2.7433749716365696e-12im -1.0167144903761027e-12 - 1.0667751404458414e-12im 0.9999999962089026 + 4.1470421878276345e-19im]Test Summary:                     | Pass  Total     Time
Trace preserving condition in QPT |    1      1  1m32.5s


Test.DefaultTestSet("Trace preserving condition in QPT", Any[], 1, false, false, true, 1.690718730347e9, 1.690718822843e9, false)

In [6]:
#"Choi POVM matrix generation (Which is interesting to generate the ideal one)"

"Choi POVM matrix generation (Which is interesting to generate the ideal one)"

In [7]:
@testset "Choi POVM matrix" begin
  N = 2
  d = 2^(2 * N)
  nshots = 3
  gates = randomcircuit(N; depth=2)

  Λ = runcircuit(gates; process=true, noise=("DEP", (p=0.001,)))
  preps = fullpreparations(N)
  bases = fullbases(N)
  data = getsamples(Λ, preps, bases, nshots)
  
  display(Λ)
  Λmat = PastaQ.array(Λ)
  display(Λmat)
  Λvec = vec(Λmat)
  display(Λvec)

  probs = PastaQ.empirical_probabilities(data)
  display(probs)

  A = PastaQ.design_matrix(probs; return_probs=false, process=true)#
  display(A)

  real_probs = A * Λvec
  Λ̂vec = pinv(A) * real_probs
  Λ̂ = reshape(Λ̂vec, (d, d))
  @test Λmat ≈ Λ̂
end

MPO
[1] ((dim=2|id=699|"Output,Qubit,Site,n=1")', (dim=2|id=699|"Input,Qubit,Site,n=1")', (dim=2|id=699|"Input,Qubit,Site,n=1"), (dim=2|id=699|"Output,Qubit,Site,n=1"), (dim=16|id=347|"Link,n=1"))
[2] ((dim=16|id=347|"Link,n=1"), (dim=2|id=938|"Output,Qubit,Site,n=2")', (dim=2|id=938|"Input,Qubit,Site,n=2")', (dim=2|id=938|"Input,Qubit,Site,n=2"), (dim=2|id=938|"Output,Qubit,Site,n=2"))


16×16 Matrix{ComplexF64}:
  0.0223131-2.17749e-16im  …  -0.0327512+0.0818046im
 0.00525756+0.125749im          -0.48021-0.169344im
  0.0559976-0.0259519im        0.0132691+0.249348im
  0.0475101+0.0302805im        -0.185174+0.132912im
 0.00488541-0.0430422im         0.154318+0.0830728im
 -0.0397098-0.0443346im    …    0.226231-0.0824806im
  0.0471282+0.0846894im        -0.388956+0.0496612im
  0.0701614-0.0208397im       -0.0272306+0.294859im
 -0.0747876-0.0666008im         0.362608-0.180748im
 -0.0268152+0.0413275im        -0.114901-0.162861im
 -0.0678064+0.0531363im    …  -0.0976148-0.334579im
  0.0170108-0.00166892im      -0.0193112+0.0664013im
  0.0377276+0.0527706im        -0.254935+0.0623501im
 -0.0352955+0.0618375im        -0.179183-0.225554im
 -0.0271547+0.0742852im        -0.238178-0.213696im
 -0.0327512-0.0818046im    …    0.357035+1.72605e-16im

256-element Vector{ComplexF64}:
   0.02231312177389383 - 2.1774873620120754e-16im
 0.0052575631049269714 + 0.12574854793199217im
   0.05599764275921061 - 0.025951948464004164im
   0.04751010711225735 + 0.030280521992698133im
  0.004885414050270766 - 0.04304219473987845im
  -0.03970979276381297 - 0.044334605207493216im
   0.04712815080474611 + 0.08468940158017196im
   0.07016140536833959 - 0.020839708616552384im
  -0.07478757325248499 - 0.06660081319210624im
 -0.026815219393898495 + 0.0413275172245548im
   -0.0678063757382372 + 0.05313631799680475im
  0.017010848374858206 - 0.0016689248347860118im
    0.0377275596200684 + 0.052770623685218725im
                       ⋮
   0.15431754959901686 + 0.08307280284670826im
   0.22623054865434103 - 0.08248064298669858im
   -0.3889558444314604 + 0.04966115573066174im
 -0.027230562161878046 + 0.2948590750502604im
   0.36260828812054346 - 0.18074846075084214im
  -0.11490099943853264 - 0.16286141365890872im
     -0.09761483842794 - 0.334578843014948

Dict{Tuple, Dict{Tuple, Float64}} with 81 entries:
  ("Z", "Y", "Y", "Z") => Dict((1, 1, 1, 0)=>0.0833333, (0, 1, 1, 0)=>0.0833333…
  ("X", "Y", "X", "Y") => Dict((1, 1, 1, 0)=>0.0, (0, 1, 1, 0)=>0.0, (1, 0, 0, …
  ("Y", "Z", "Y", "Y") => Dict((1, 1, 1, 0)=>0.0, (0, 1, 1, 0)=>0.166667, (1, 0…
  ("Y", "Y", "Z", "Z") => Dict((1, 1, 1, 0)=>0.166667, (1, 0, 0, 1)=>0.0833333,…
  ("Z", "X", "X", "Y") => Dict((1, 1, 1, 0)=>0.0, (0, 1, 1, 0)=>0.0, (1, 0, 0, …
  ("Z", "Z", "X", "X") => Dict((1, 1, 1, 0)=>0.0833333, (0, 1, 1, 0)=>0.0833333…
  ("X", "Z", "X", "Z") => Dict((1, 1, 1, 0)=>0.0, (0, 1, 1, 0)=>0.0, (1, 0, 0, …
  ("Z", "X", "Z", "Y") => Dict((1, 1, 1, 0)=>0.0, (0, 1, 1, 0)=>0.0833333, (1, …
  ("Y", "Y", "Y", "Y") => Dict((1, 1, 1, 0)=>0.0833333, (0, 1, 1, 0)=>0.0833333…
  ("Z", "Z", "Y", "X") => Dict((1, 1, 1, 0)=>0.166667, (0, 1, 1, 0)=>0.166667, …
  ("X", "Z", "Z", "Z") => Dict((1, 1, 1, 0)=>0.166667, (0, 1, 1, 0)=>0.25, (1, …
  ("X", "Z", "Y", "Z") => Dict((1, 1, 1, 0)=>0.166667, (0,

1296×256 Matrix{ComplexF64}:
    0.0-0.0im  0.0-0.0im     0.0+0.0im     …  0.0-0.0im        0.0-0.0im
   0.25-0.0im  0.0-0.0im     0.0+0.25im       0.0-0.0im        0.0-0.0im
    0.0-0.0im  0.0-0.0im     0.0-0.0im        0.0-0.0im       0.25-0.0im
    0.0-0.0im  0.0-0.0im     0.0-0.0im        0.0-0.0im        0.0-0.0im
    0.0+0.0im  0.0+0.0im     0.0-0.0im        0.0+0.0im       0.25+0.0im
    0.0+0.0im  0.0+0.0im     0.0-0.0im     …  0.0+0.0im        0.0+0.0im
    0.0-0.0im  0.0-0.0im     0.0-0.0im        0.0-0.0im        0.0-0.0im
   0.25-0.0im  0.0-0.0im     0.0-0.25im       0.0-0.0im        0.0-0.0im
    0.0-0.0im  0.0-0.0im     0.0-0.0im        0.0-0.0im       0.25-0.0im
    0.0+0.0im  0.0+0.0im     0.0-0.0im        0.0+0.0im        0.0+0.0im
    0.0-0.0im  0.0-0.0im     0.0+0.0im     …  0.0-0.0im        0.0-0.0im
   0.25+0.0im  0.0+0.0im     0.0-0.25im       0.0+0.0im        0.0+0.0im
    0.0-0.0im  0.0-0.0im     0.0+0.0im        0.0-0.0im       0.25-0.0im
       ⋮              

Test Summary:    | Pass  Total  Time
Choi POVM matrix |    1      1  4.4s


Test.DefaultTestSet("Choi POVM matrix", Any[], 1, false, false, true, 1.690718823486e9, 1.690718827889e9, false)

In [8]:

function stomography(
  data::Matrix{Pair{String,Int}};
  method::String="linear_inversion",
  fillzeros::Bool=true,
  kwargs...,
)
  return stomography(
    empirical_probabilities(data; fillzeros=fillzeros),
    siteinds("Qubit", size(data, 2));
    method=method,
    kwargs...,
  )
end

function stomography(
  data::Matrix{Pair{String,Int}},
  sites::Vector{<:Index};
  method::String="linear_inversion",
  fillzeros::Bool=true,
  kwargs...,
)
  return stomography(
    empirical_probabilities(data; fillzeros=fillzeros), sites; method=method, kwargs...
  )
end

function stomography(data::Matrix{Pair{String,Pair{String,Int}}}; kwargs...)
  return stomography(data, siteinds("Qubit", size(data, 2)); kwargs...)
end

function stomography(
  data::Matrix{Pair{String,Pair{String,Int}}},
  sites::Vector{<:Index};
  method::String="linear_inversion",
  fillzeros::Bool=true,
  kwargs...,
)
  sites_in = addtags.(sites, "Input")
  sites_out = addtags.(sites, "Output")
  process_sites = Index[]
  for j in 1:size(data, 2)
    push!(process_sites, sites_in[j])
    push!(process_sites, sites_out[j])
  end
  return stomography(
    empirical_probabilities(data; fillzeros=fillzeros),
    process_sites;
    method=method,
    process=true,
    kwargs...,
  )
end


function stomography(train_data::AbstractMatrix, L::LPDO; (observer!)=nothing, kwargs...)
  # Read arguments
  opt = get(kwargs, :optimizer, Optimisers.Descent(0.01))
  batchsize::Int64 = get(kwargs, :batchsize, 100)
  epochs::Int64 = get(kwargs, :epochs, 1000)
  trace_preserving_regularizer = get(kwargs, :trace_preserving_regularizer, 0.0)
  observe_step::Int64 = get(kwargs, :observe_step, 1)
  test_data = get(kwargs, :test_data, nothing)
  print_metrics = get(kwargs, :print_metrics, [])
  outputpath = get(kwargs, :outputpath, nothing)
  outputlevel = get(kwargs, :outputlevel, 1)
  savestate = get(kwargs, :savestate, false)
  earlystop = get(kwargs, :earlystop, false)

  model = copy(L)
  isqpt = train_data isa Matrix{Pair{String,Pair{String,Int}}}
  localnorm = isqpt ? 2.0 : 1.0
  print(" ")
  print(test_data)
  print(" ")
  print("Bananas:")
  println()

  # observer is not passed but earlystop is called
  observer! = (isnothing(observer!) || earlystop) ? Observer() : observer!

  # observer is defined
  if !isnothing(observer!)
    observer!["train_loss"] = nothing
    if !isnothing(test_data)
      observer!["test_loss"] = nothing
    end
    # add the standard early stop function to the observer
    if earlystop
      function stop_if(; loss::Vector)
        return stoptomography_ifloss(; loss=loss, ϵ=1e-3, min_iter=50, window=50)
      end
      observer!["earlystop"] = stop_if
    end
  end

  # initialize optimizer
  st = PastaQ.state(opt, model)
  optimizer = (opt, st)

  @assert size(train_data, 2) == length(model)
  !isnothing(test_data) && @assert size(test_data)[2] == length(model)

  batchsize = min(size(train_data)[1], batchsize)
  num_batches = Int(floor(size(train_data)[1] / batchsize))

  tot_time = 0.0
  observe_time = 0.0
  best_model = nothing
  best_testloss = 1000.0
  test_loss = nothing

  # Training iterations
  for ep in 1:epochs
    ep_time = @elapsed begin
      train_data = train_data[shuffle(1:end), :]
      train_loss = 0.0

      # Sweep over the data set
      for b in 1:num_batches
        batch = train_data[((b - 1) * batchsize + 1):(b * batchsize), :]

        normalized_model = copy(model)
        sqrt_localnorms = []
        PastaQ.normalize!(
          normalized_model; (sqrt_localnorms!)=sqrt_localnorms, localnorm=localnorm
        )

        grads, loss = PastaQ.gradients(
          normalized_model,
          batch;
          sqrt_localnorms=sqrt_localnorms,
          trace_preserving_regularizer=trace_preserving_regularizer,
        )

        nupdate = ep * num_batches + b
        train_loss += loss / Float64(num_batches)
        update!(model, grads, optimizer)
      end
    end # end @elapsed
    !isnothing(observer!) && push!(last(observer!["train_loss"]), train_loss)
    observe_time += ep_time

    # measurement stage
    if ep % observe_step == 0
      normalized_model = copy(model)
      sqrt_localnorms = []
      PastaQ.normalize!(normalized_model; (sqrt_localnorms!)=sqrt_localnorms, localnorm=localnorm)

      if !isnothing(test_data)
        test_loss = nll(normalized_model, test_data)
        !isnothing(observer!) && push!(last(observer!["test_loss"]), test_loss)
        if test_loss ≤ best_testloss
          best_testloss = test_loss
          best_model = copy(normalized_model)
        end
      else
        best_model = copy(normalized_model)
      end

      # update observer
      if !isnothing(observer!)
        loss = (
          if !isnothing(test_data)
            results(observer!, "test_loss")
          else
            results(observer!, "train_loss")
          end
        )

        model_to_observe = (
          if isqpt && (normalized_model isa LPDO{MPS})
            PastaQ.choi_mps_to_unitary_mpo(normalized_model)
          elseif !isqpt && (normalized_model isa LPDO{MPS})
            normalized_model.X
          else
            normalized_model
          end
        )
        update!(
          observer!, model_to_observe; train_loss=train_loss, test_loss=test_loss, loss=loss
        )
        tot_time += observe_time
      end

      # printing
        
      if outputlevel ≥ 1
        #print("%d  ", ep)
        print(train_loss)
        print(",")
        print("testloss:")
        print(test_loss)
        if !isnothing(test_data)
          print("(%d)  ", test_loss)
        end
        # TODO: add the trace preserving cost function here for QPT
        !isnothing(observer!) && PastaQ.printobserver(observer!, print_metrics)
        #print("elapsed = %d", observe_time)
        observe_time = 0.0
        print("")
      end
      # saving
      if !isnothing(outputpath)
        observerpath = outputpath * "_observer.jld2"
        save(observerpath, observer!)
        if savestate
          if isqpt
            model_to_be_saved =
              model isa LPDO{MPS} ? PastaQ.choi_mps_to_unitary_mpo(best_model) : best_model
          else
            model_to_be_saved = model isa LPDO{MPS} ? best_model.X : best_model
          end
          statepath = outputpath * "_state.h5"
          h5rewrite(statepath) do fout
            write(fout, "state", model_to_be_saved)
          end
        end
      end
    end
    !isnothing(observer!) &&
      haskey(observer!.data, "earlystop") &&
      results(observer!, "earlystop")[end] &&
      break
  end
  return best_model
end

# QST
function stomography(data::Matrix{Pair{String,Int}}, ψ::MPS; kwargs...)
  return stomography(data, LPDO(ψ); kwargs...).X
end

function stomography(train_data::Vector{<:Vector{Pair{String,Int}}}, args...; kwargs...)
  return stomography(permutedims(hcat(train_data...)), args...; kwargs...)
end

# QPT###################################################################################### passes here
function stomography(data::Matrix{Pair{String,Pair{String,Int}}}, U::MPO; kwargs...)
  return PastaQ.choi_mps_to_unitary_mpo(
    stomography(data, LPDO(PastaQ.unitary_mpo_to_choi_mps(U)); kwargs...)
  )
end

function stomography(
  train_data::Vector{<:Vector{Pair{String,Pair{String,Int}}}}, args...; kwargs...
)
  return stomography(permutedims(hcat(train_data...)), args...; kwargs...)
end

"""
EARLY STOPPING FUNCTIONS
"""

#stopif_fidelity(M1, M2; ϵ::Number, kwargs...)
#  fidelity(M1,M2) ≤ ϵ

function stoptomography_ifloss(; loss::Vector, ϵ::Number, min_iter::Number, window::Number)
  length(loss) < min_iter + 1 && return false
  length(loss) < window && return false
  avgloss = StatsBase.mean(loss[(end - window):end])
  Δ = StatsBase.sem(loss[(end - window):end])
  return Δ / avgloss < ϵ
end

stoptomography_ifloss (generic function with 1 method)

In [2]:
using PastaQ
using Random
using ITensors
using Observers
using Optimisers: Optimisers
using Test
using LinearAlgebra
using SCS
using Convex

N = 2
nshots = 1
gates = randomcircuit(N; depth=10)
Λ1 = runcircuit(gates; process=true)

print("Run circuit MPO:")
display(Λ1)

#For training data
preps = randompreparations(N,4) #random initialization of states
bases = randombases(N,4)        #random basis of measurements
data = getsamples(Λ1, preps, bases,100) #Get samples for these datapoints accordingly to the preparation and basis

#For testing data
preps = randompreparations(N,4)
bases = randombases(N,4)
target = getsamples(Λ1, preps, bases, 2)

print("Part 1:----------------------------------")
#display(data)
print("-----------------------------------------")

χ = maxlinkdim(Λ1) #
U0 = randomprocess(Λ1,mixed=false; χ=χ) #definition of the variational circuit
print("This is U0:")
display(U0)
print("Part 2---------------------------------------------")

#F(bu::LPDO; kwargs...) = fidelity(bu, Λ; process=true)
#obs = Observer(["F" => F])

print("Part 3---------------------------------------------")

N = size(data, 2) # Number of qubits

opt = Optimisers.ADAM() #optimizer
#opt=Optimisers.Descent(0.01)

# Run quantum state tomography
Λ = tomography(data, U0; optimizer = opt, test_data = target,target=Λ1, batchsize=500,epochs=100) #observer=obs,
#display(Λ)




Run circuit MPO:

MPO
[1] ((dim=2|id=518|"Qubit,Site,n=1")', (dim=2|id=518|"Qubit,Site,n=1"), (dim=4|id=866|"Link,n=1"))
[2] ((dim=4|id=866|"Link,n=1"), (dim=2|id=392|"Qubit,Site,n=2")', (dim=2|id=392|"Qubit,Site,n=2"))


Part 1:---------------------------------------------------------------------------This is U0:

MPO
[1] ((dim=2|id=518|"Qubit,Site,n=1")', (dim=4|id=839|"Link,l=1"), (dim=2|id=518|"Qubit,Site,n=1"))
[2] ((dim=2|id=392|"Qubit,Site,n=2")', (dim=4|id=839|"Link,l=1"), (dim=2|id=392|"Qubit,Site,n=2"))


Part 2---------------------------------------------Part 3---------------------------------------------1     ⟨logP⟩ = 3.2787  (1.6531)  elapsed = 18.507s
2     ⟨logP⟩ = 3.1294  (1.6580)  elapsed = 0.225s
3     ⟨logP⟩ = 3.0074  (1.6693)  elapsed = 0.200s
4     ⟨logP⟩ = 2.8857  (1.6980)  elapsed = 0.175s
5     ⟨logP⟩ = 2.7772  (1.7921)  elapsed = 0.170s
6     ⟨logP⟩ = 2.6707  (1.8855)  elapsed = 0.159s
7     ⟨logP⟩ = 2.5860  (1.7371)  elapsed = 0.185s
8     ⟨logP⟩ = 2.5310  (1.7100)  elapsed = 0.159s
9     ⟨logP⟩ = 2.4777  (1.7189)  elapsed = 0.182s
10    ⟨logP⟩ = 2.4398  (1.7273)  elapsed = 0.180s
11    ⟨logP⟩ = 2.4221  (1.7165)  elapsed = 0.167s
12    ⟨logP⟩ = 2.3985  (1.7171)  elapsed = 0.164s
13    ⟨logP⟩ = 2.3743  (1.7218)  elapsed = 0.166s
14    ⟨logP⟩ = 2.3651  (1.7318)  elapsed = 0.165s
15    ⟨logP⟩ = 2.3670  (1.7466)  elapsed = 0.167s
16    ⟨logP⟩ = 2.3595  (1.7626)  elapsed = 0.159s
17    ⟨logP⟩ = 2.3414  (1.7839)  elapsed = 0.163s
18    ⟨logP⟩ = 2.3502  (1.7772)  elapsed = 0.15

MPO
[1] ((dim=2|id=518|"Qubit,Site,n=1")', (dim=4|id=839|"Link,l=1"), (dim=2|id=518|"Qubit,Site,n=1"))
[2] ((dim=2|id=392|"Qubit,Site,n=2")', (dim=4|id=839|"Link,l=1"), (dim=2|id=392|"Qubit,Site,n=2"))


In [3]:
#Λ1,Λ - quantum circuit tensor network and variational tensor network

#Variational vector format
Λmat = PastaQ.array(Λ)
Λvec = vec(Λmat)

print("Variational tensor network:")
print(" ")
display(Λvec)

#Random circuit vector format
Λmat1 = PastaQ.array(Λ1)
Λvec1 = vec(Λmat1)


print("Quantum circuit:")
print(" ")
display(Λvec1)

MSE=0
for i in 1:size(Λvec, 1)
    MSE=abs(Λvec[i]*Λvec[i]-Λvec1[i]*Λvec1[i]) + MSE
end 

#MSE=MSE/size(Λvec, 1)

print("MSE non normalized:")
display(MSE)


Variational tensor network: 

16-element Vector{ComplexF64}:
    0.04619703643647061 - 0.6164726597758794im
   -0.49293715791112075 - 0.5777797980893581im
      -0.55076066551705 + 0.5699259783582641im
   0.050482928973565794 + 0.15761664641343726im
     0.3320597997419309 - 0.7334602570113685im
   -0.20918853973231027 - 0.20823046157166908im
   -0.22597955672626108 + 0.11561929028525401im
    0.10862808821071185 + 0.18191582991223038im
   -0.12428496842281095 - 0.27262433138216025im
   0.019348165385902637 + 0.22294901168193784im
 -0.0034558009363883646 + 0.236596618322161im
   -0.17272778073670614 - 0.4203133796558753im
     0.6046556346250295 + 0.530881710110966im
    0.04514434870631548 - 0.27297869615628534im
   -0.11715146035410286 - 0.2580195420968451im
    0.22939523553546565 + 0.5305953286004398im

Quantum circuit: 

16-element Vector{ComplexF64}:
    0.08180867575003772 + 0.022237863339273342im
      0.789712492678016 - 0.08388240789701462im
   -0.16336210488290992 - 0.30726264264615455im
   -0.30241868895141855 - 0.3867507805487037im
     0.4192013751559086 + 0.8394372381604367im
    -0.1379403316805653 - 0.2204475366495596im
    0.08294744225997683 - 0.037346101907064305im
    0.10235779066034822 - 0.1823144314747876im
 -0.0023252943484326325 + 0.26224992912741707im
  -0.004144539115334855 + 0.3489834535958693im
    0.10039652542826079 - 0.38172407698465627im
    -0.5328899874552204 + 0.6079872265046382im
   -0.06925076924603449 + 0.1971092258134826im
   -0.14404965406521997 + 0.39891896183332853im
    -0.7829531738666449 + 0.31909101504350623im
    -0.1936462477886211 - 0.155344859842848im

MSE non normalized:

6.64233604017714

In [11]:
for i in 1:10
    println("9 * $i = ",9*i)
end

9 * 1 = 9
9 * 2 = 18
9 * 3 = 27
9 * 4 = 36
9 * 5 = 45
9 * 6 = 54
9 * 7 = 63
9 * 8 = 72
9 * 9 = 81
9 * 10 = 90


In [12]:
using ITensors
function main()
    a = Index(3,"a")
    @show a
    b = Index(2,"b")
    c = Index(4,"c")
    d = Index(5,"d")
    i = Index(2,"i")
    j = Index(6,"j")
    A = randomITensor(a,b,d,c)
    B = randomITensor(i,d,j)
    
    C = A * B
    @show hasinds(C,a,b,c,i,j)
    @show C
    return C
end
main()

a = (dim=3|id=650|"a")
hasinds(C, a, b, c, i, j) = true
C = ITensor ord=5
Dim 1: (dim=3|id=650|"a")
Dim 2: (dim=2|id=684|"b")
Dim 3: (dim=4|id=274|"c")
Dim 4: (dim=2|id=592|"i")
Dim 5: (dim=6|id=110|"j")
NDTensors.Dense{Float64, Vector{Float64}}
 3×2×4×2×6
[:, :, 1, 1, 1] =
  0.7480220297079663   1.145704070939422
 -0.826344480858588   -1.1465923533586044
  0.3858421121494405   0.3027515922530699

[:, :, 2, 1, 1] =
  0.6576731256531949   4.055911860244057
 -2.6284339811853514   1.2600416120777944
  1.1356087722707142  -3.8132520273545127

[:, :, 3, 1, 1] =
 0.3010359475553373  0.8536598538370233
 0.5354077430400139  4.450542505996551
 7.894038252344049   2.1339642612187477

[:, :, 4, 1, 1] =
  2.2378611592135766   0.46324611566220547
 -1.8307879443393842  -3.258738466660836
 -3.842097728015382    0.24755921640331735

[:, :, 1, 2, 1] =
  0.0860566330450529   -2.033552559987741
 -0.8433938621721965   -1.0229837240682216
  0.41921310572563425  -0.7406398013092955

[:, :, 2, 2, 1] =
 1.304

ITensor ord=5 (dim=3|id=650|"a") (dim=2|id=684|"b") (dim=4|id=274|"c") (dim=2|id=592|"i") (dim=6|id=110|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [13]:
#Working version with tomography using variational tensor networks:

In [14]:
using PastaQ
using Random
using ITensors
using Observers
using Optimisers: Optimisers


N = 3
depth = 4
nshots = 3

gates = randomcircuit(N; depth=4)
print("1")

Λ = runcircuit(gates; process=true)
print("2")

preps = randompreparations(N,nshots)
print("3")

bases = randombases(N,nshots)
print("4")

data = getsamples(Λ, preps, bases, nshots)

print("5")

data, target = split_dataset(data; train_ratio=0.9)
print("6")


χ = maxlinkdim(ρ) #
U0 = randomprocess(ρ; χ=χ) #definition of the variational circuit
display(U0)
print("7")

N = size(data, 2) # Number of qubits

opt = Optimisers.ADAM() #optimizer

# Set parameters
##N = length(Û)     # Number of qubits
##χ = maxlinkdim(Û) # Bond dimension of variational MPS
##U0 = randomprocess(Û; χ=χ)


# Run quantum state tomography
Λ = tomography(data, U0; optimizer = opt, target = target)
display(Λ)

for j in 1:N
    s = firstind(ρ; tags="Output,n=$(j)", plev=0)
    ρ = ρ * δ(s, s')
  end

ρmat = PastaQ.array(ρ)
display(ρmat)

12345

LoadError: InexactError: Int64(24.3)

In [ ]:
using PastaQ
using ITensors
using Random

Random.seed!(1234)

N = 4     # Number of qubits
depth = 4 # Depth of the quantum circuit

# Generate random quantum circuit built out of
# layers of single-qubit random rotations + `CX` 
# gates, alternating between even and of odd layers.
println("Random circuit of depth $depth on $N qubits:")
circuit = randomcircuit(N, depth=depth; twoqubitgates="CX", onequbitgates="Rn")
display(circuit)
println()

# 1. Unitary quantum circuit
# Returns the MPS at the output of the quantum circuit:
# `|ψ⟩ = Û|0,0,…,0⟩`
# where `Û` is the unitary circuit.
println("Applying random circuit to compute |ψ⟩ = U|0,0,…,0⟩...")
ψ = runcircuit(circuit)
@show maxlinkdim(ψ)
println()

# A representation of the unitary operation as a MPO
# is obtained using the flag `process=true`:
println("Approximating random circuit as an MPO U...")
U = runcircuit(circuit; process=true)
@show maxlinkdim(U)
println()

# 2. Quantum circuit with noise
# Apply a noise model `noise` (specified by appropriate 
# parameters) to each quantum gate in `gates`.
# Returns a mixed density operator as MPO:
# `ρ = ε(|0,0,…⟩⟨0,0,…|)`
# where `ε` is the quantum channel.
# Here, the noise is a single-qubit amplitude damping 
# channel with decay rate `γ=0.01`..
println(
  "Running the circuit with amplitude damping to compute the state ρ = ε(|0,0,…⟩⟨0,0,…|)..."
)
ρ = runcircuit(circuit; noise=("amplitude_damping", (γ=0.01,)))
@show maxlinkdim(ρ)
println()

# A representation of the quantum channel as a MPO
# is obtained using the flag `process=true`, which 
# returns the Choi matrix `Λ` of the channel:`:
println(
  "Running the circuit with amplitude damping to compute the Choi matrix Λ of the quantum channel...",
)
Λ = runcircuit(circuit; process=true, noise=("amplitude_damping", (γ=0.01,)))
@show maxlinkdim(Λ)

Random circuit of depth 4 on 4 qubits:


4-element Vector{Vector}:
 Any[("CX", (1, 2)), ("CX", (3, 4)), ("Rn", 1, (θ = 1.0240860966391014, ϕ = 3.449790032588226, λ = 0.6867102289516064)), ("Rn", 2, (θ = 2.8093548677424836, ϕ = 2.218665895883152, λ = 1.2385897659119816)), ("Rn", 3, (θ = 2.9943293270335696, ϕ = 4.998568891921718, λ = 1.5527317508606786)), ("Rn", 4, (θ = 2.351215134597933, ϕ = 3.633138470776425, λ = 2.286875252144381))]
 Any[("CX", (2, 3)), ("Rn", 1, (θ = 0.02339860135090042, ϕ = 1.2527202123030283, λ = 1.3799229760638079)), ("Rn", 2, (θ = 2.1442395975985904, ϕ = 6.011380704980616, λ = 2.035297500617958)), ("Rn", 3, (θ = 3.1311163576685583, ϕ = 4.7073251097507764, λ = 0.34583990610826215)), ("Rn", 4, (θ = 1.5437258380134744, ϕ = 3.550913017731251, λ = 0.7973732429466119))]
 Any[("CX", (1, 2)), ("CX", (3, 4)), ("Rn", 1, (θ = 1.9691311440778374, ϕ = 1.4709222894013594, λ = 0.3920454955798837)), ("Rn", 2, (θ = 1.915978397587573, ϕ = 4.227282390786751, λ = 2.393628962683993)), ("Rn", 3, (θ = 1.8499961361059682, ϕ = 


Applying random circuit to compute |ψ⟩ = U|0,0,…,0⟩...
maxlinkdim(ψ) = 4

Approximating random circuit as an MPO U...
maxlinkdim(U) = 4

Running the circuit with amplitude damping to compute the state ρ = ε(|0,0,…⟩⟨0,0,…|)...


┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ C:\Users\vladl\.julia\packages\PastaQ\D5CCg\src\circuits\noise.jl:177
┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ C:\Users\vladl\.julia\packages\PastaQ\D5CCg\src\circuits\noise.jl:177
┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ C:\Users\vladl\.julia\packages\PastaQ\D5CCg\src\circuits\noise.jl:177
┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ C:\Users\vladl\.julia\packages\PastaQ\D5CCg\src\circuits\noise.jl:177
┌ Warning: Noise model not defined for 2-qubit gates! Applying tensor-product noise instead.
└ @ PastaQ C:\Users\vladl\.julia\packages\PastaQ\D5CCg\src\circuits\noise.jl:177


In [ ]:
using PastaQ
using Random
using ITensors
using Observers
using Optimisers: Optimisers


N = 3
depth = 4
nshots = 100
circuit = randomcircuit(N,depth=depth)
ρ = runcircuit(circuit) #density matrix

bases= randombases(N, nshots) #basis of dimension 3 with 100 shots
print("bases:")
display(bases)


data=getsamples(ρ,bases) #sample density matrix in the basis for training and testing data
target=getsamples(ρ,bases)

χ = maxlinkdim(ρ) #
U0 = randomstate(ρ; χ=χ)
print("Random state:")
display(U0)
#display(U)

#print(circuit)
# Generate samples
#print(bases)
#data,Û= getsamples(ρ, bases,nshots)
##data = getsamples(ρ, nshots)
#target = getsamples(ρ, nshots)

#display(data)

#writesamples(data, U, "data/qpt_circuit.h5")

# Load target state and measurements. Each samples is built out
# of an input state (`first.(data)`) to the quantum channel, and the
# measurement output (`last.(data)`) after a local basis rotation.
#data, Û = readsamples("data/qpt_circuit.h5")

# Split data into a train and test dataset

#data, target = split_dataset(data; train_ratio=0.9)

display(data)
display(target)

# Load the training data, as well as the target quantum state from file.
#data, target = loadsamples("PATH_TO_DATAFILE.h5")
N = size(data, 2) # Number of qubits
print("N:")
display(N)
# 1. Reconstruction with a variational MPO:
#

# Initialize stochastic gradient descent with learning rate η = 0.01
opt = Optimisers.ADAM()

# Run quantum state tomography
#U = tomography(data, U0; optimizer = opt, target = target)

# 2. Reconstruction with a variational density matrix:
#
# Initialize a variational LPDO with bond dimension χ = 10 and Kraus dimension ξ = 2.
Λ0 = randomprocess(N; mixed = true, χ = 10, ξ = 3)

# Set parameters
##N = length(Û)     # Number of qubits
##χ = maxlinkdim(Û) # Bond dimension of variational MPS
##U0 = randomprocess(Û; χ=χ)

display(Λ0)

# Run quantum state tomography
Λ = tomography(data, U0; optimizer = opt, target = target)
display(Λ)

In [ ]:
N = 3
  χ = 4
  nsamples = 10
  trace_preserving_regularizer = 0.1
  Random.seed!(1234)
  data = randompreparations(N, nsamples)
  data_out = PastaQ.convertdatapoints(randompreparations(N, nsamples))
  data = data_in .=> data_out

In [ ]:
#Test full tomography

In [ ]:
using ITensors
using PastaQ
using Test
using LinearAlgebra
using Random

using SCS
using Convex

@testset " counts and frequencies" begin
  N = 3
  d = 1 << N
  gates = randomcircuit(N; depth=4)
  ψ = runcircuit(N, gates)

  bases = fullbases(N)
  samples = getsamples(ψ, bases, 100)

  C = PastaQ.measurement_counts(samples)
  @test length(keys(C)) == 3^N
  probs1 = PastaQ.empirical_probabilities(samples)
  @test length(keys(probs1)) == 3^N
  probs2 = PastaQ.empirical_probabilities(C)
  @test length(keys(probs2)) == 3^N
  @test probs1 == probs2

  for (basis, counts_in_basis) in C
    tot_counts = sum(values(counts_in_basis))
    for (proj, counts) in counts_in_basis
      freq = counts / tot_counts
      @test freq ≈ probs1[basis][proj]
    end
  end
end

@testset "POVM matrix" begin
  N = 3
  d = 1 << N
  gates = randomcircuit(N, ; depth=4)
  ψ = runcircuit(N, gates)

  bases = fullbases(N)
  samples = getsamples(ψ, bases, 100)

  ρ = outer(ψ', ψ)
  ρmat = PastaQ.array(ρ)
  #ρ = projector(toarray(ψ))
  ρ_vec = vec(ρmat)

  probs = PastaQ.empirical_probabilities(samples)
  A = PastaQ.design_matrix(probs; return_probs=false)
  real_probs = A * ρ_vec
  ρ̂_vec = pinv(A) * real_probs
  ρ̂ = reshape(ρ̂_vec, (d, d))
  @test ρmat ≈ ρ̂
end

@testset "make PSD" begin
  ρ = zeros(5, 5)
  ρ[1, 1] = 3 / 5
  ρ[2, 2] = 1 / 2
  ρ[3, 3] = 7 / 20
  ρ[4, 4] = 1 / 10
  ρ[5, 5] = -11 / 20

  ρ̂ = PastaQ.make_PSD(ρ)
  λ = reverse(first(eigen(ρ̂)))
  @test λ[1] ≈ 9 / 20
  @test λ[2] ≈ 7 / 20
  @test λ[3] ≈ 1 / 5
  @test λ[4] ≈ 0.0
  @test λ[5] ≈ 0.0
end

@testset "PSD constraint in QST" begin
  N = 2
  d = 1 << N
  gates = randomcircuit(N; depth=4)
  ψ = runcircuit(N, gates)

  bases = fullbases(N)
  samples = getsamples(ψ, bases, 100)

  ϱ = PastaQ.array(outer(ψ', ψ))

  ρ = PastaQ.array(tomography(samples; method="LI"))
  λ = first(eigen(ρ))
  @test all(λ .≥ -1e-4)

  ρ = PastaQ.array(tomography(samples; method="LS"))
  λ = first(eigen(ρ))
  @test all(real(λ) .≥ -1e-3)

  ρ = PastaQ.array(tomography(samples; method="MLE"))
  λ = first(eigen(ρ))
  @test all(real(λ) .≥ -1e-2)
end

@testset "arbitrary trace in QST" begin
  N = 2
  d = 1 << N
  gates = randomcircuit(N; depth=4)
  ψ = runcircuit(N, gates)

  bases = fullbases(N)
  samples = getsamples(ψ, bases, 100)

  ϱ = PastaQ.array(outer(ψ', ψ))

  ρ = PastaQ.array(tomography(samples; method="LI", trρ=2.0))
  @test tr(ρ) ≈ 2.0
  ρ = PastaQ.array(tomography(samples; method="LS", trρ=2.0))
  @test tr(ρ) ≈ 2.0 atol = 1e-4
  ρ = PastaQ.array(tomography(samples; method="MLE", trρ=2.0))
  @test tr(ρ) ≈ 2.0 atol = 1e-4
end

@testset "Choi POVM matrix" begin
  N = 2
  d = 2^(2 * N)
  nshots = 3
  gates = randomcircuit(N; depth=2)

  Λ = runcircuit(gates; process=true, noise=("DEP", (p=0.001,)))
  preps = fullpreparations(N)
  bases = fullbases(N)
  data = getsamples(Λ, preps, bases, nshots)

  Λmat = PastaQ.array(Λ)
  Λvec = vec(Λmat)

  probs = PastaQ.empirical_probabilities(data)
  A = PastaQ.design_matrix(probs; return_probs=false, process=true)
  real_probs = A * Λvec
  Λ̂vec = pinv(A) * real_probs
  Λ̂ = reshape(Λ̂vec, (d, d))
  @test Λmat ≈ Λ̂
end

@testset "PSD constraint in QPT" begin
  N = 2
  d = 1 << N
  nshots = 3
  gates = randomcircuit(N; depth=4)
  Λ = runcircuit(gates; process=true, noise=("DEP", (p=0.001,)))
  preps = fullpreparations(N)
  bases = fullbases(N)
  data = getsamples(Λ, preps, bases, nshots)

  ρ = PastaQ.array(tomography(data; method="LI"))
  λ = first(eigen(ρ))
  @test all(λ .≥ -1e-4)

  ρ = PastaQ.array(tomography(data; method="LS"))
  λ = first(eigen(ρ))
  @test all(real(λ) .≥ -1e-4)
end

@testset "Trace preserving condition in QPT" begin
  Random.seed!(1234)
  N = 2
  d = 1 << N
  nshots = 3
  gates = randomcircuit(N; depth=4)
  Λ = runcircuit(gates; process=true, noise=("DEP", (p=0.001,)))
  preps = fullpreparations(N)
  bases = fullbases(N)
  data = getsamples(Λ, preps, bases, nshots)

  ρ = tomography(data; method="LS")
  for j in 1:N
    s = firstind(ρ; tags="Output,n=$(j)", plev=0)
    ρ = ρ * δ(s, s')
  end

  ρmat = PastaQ.array(ρ)
  @test ρmat ≈ Matrix{Float64}(I, d, d) atol = 1e-5
end

In [ ]:
#Testing get samples

In [ ]:
using PastaQ
using Random
using ITensors
using Test
using LinearAlgebra

#
# Helper functions for tests
#

function state_to_int(state::Array)
  index = 0
  for j in 1:length(state)
    index += 2^(j - 1) * state[length(state) + 1 - j]
  end
  return index
end

function empiricalprobability(samples::Matrix)
  prob = zeros((1 << size(samples)[2]))
  for n in 1:size(samples)[1]
    sample = samples[n, :]
    index = state_to_int(sample)
    prob[index + 1] += 1
  end
  prob = prob / size(samples)[1]
  return prob
end

@testset "generation of preparation states" begin
  N = 4
  nstates = 100
  states = PastaQ.randompreparations(N, nstates)
  @test size(states)[1] == nstates
  @test size(states)[2] == N
end

@testset "generation of measurement bases" begin
  N = 4
  nbases = 100
  bases = randombases(N, nbases)
  @test size(bases)[1] == nbases
  @test size(bases)[2] == N

  bases = fullbases(N)
  @test bases isa Matrix{String}
  @test size(bases) == (3^N, N)
end

@testset "measurements" begin
  Random.seed!(1234)
  N = 3
  depth = 4
  ψ0 = productstate(N)
  gates = randomcircuit(N; depth=depth)
  ψ = runcircuit(ψ0, gates)
  ψ_vec = PastaQ.array(ψ)
  probs = abs2.(ψ_vec)

  nshots = 50000
  samples = PastaQ.getsamples(ψ, nshots)
  @test size(samples)[1] == nshots
  @test size(samples)[2] == N
  data_prob = empiricalprobability(samples)
  @test probs ≈ data_prob atol = 1e-2

  ρ = runcircuit(N, gates; noise=("depolarizing", (p=0.01,)))
  ρ_mat = PastaQ.array(ρ)
  probs = real(diag(ρ_mat))

  samples = PastaQ.getsamples(ρ, nshots)
  @test size(samples)[1] == nshots
  @test size(samples)[2] == N
  data_prob = empiricalprobability(samples)
  @test probs ≈ data_prob atol = 1e-2
end

@testset "basis rotations" begin
  s = siteinds("Qubit", 1)
  #ϕ = qubits(1)
  ψ0 = state("X+", s[1])
  gates = [("basisX", 1, (dag=true,))]
  ψ = runcircuit(ψ0, gates)
  @test PastaQ.array(ψ) ≈ array(state("Z+", s[1]))
  ψ0 = state("X-", s[1])
  gates = [("basisX", 1, (dag=true,))]
  ψ = runcircuit(ψ0, gates)
  @test PastaQ.array(ψ) ≈ array(state("Z-", s[1]))

  ψ0 = state("Y+", s[1])
  gates = [("basisY", 1, (dag=true,))]
  ψ = runcircuit(ψ0, gates)
  @test PastaQ.array(ψ) ≈ array(state("Z+", s[1]))
  ψ0 = state("Y-", s[1])
  gates = [("basisY", 1, (dag=true,))]
  ψ = runcircuit(ψ0, gates)
  @test PastaQ.array(ψ) ≈ array(state("Z-", s[1]))
end

@testset "project unitary MPO" begin
  N = 4
  ntrial = 100
  gates = randomcircuit(N; depth=4, layered=false)

  U = runcircuit(N, gates; process=true)

  bases = randombases(N, ntrial)
  preps = randompreparations(N, ntrial)

  for n in 1:ntrial
    mgates = PastaQ.measurementgates(bases[n, :])
    ψ_in = productstate(N, preps[n, :])
    ψ_out = runcircuit(ψ_in, gates)

    Ψ_out = PastaQ.projectchannel(U, preps[n, :])
    @test PastaQ.array(ψ_out) ≈ PastaQ.array(Ψ_out)

    ψ_m = runcircuit(ψ_out, mgates)
    Ψ_m = runcircuit(Ψ_out, mgates)
    @test PastaQ.array(ψ_m) ≈ PastaQ.array(Ψ_m)
  end
end

@testset "project unitary ITensor" begin
  N = 4
  ntrial = 100
  gates = randomcircuit(N; depth=4, layered=false)

  U = runcircuit(N, gates; process=true, full_representation=true)
  bases = randombases(N, ntrial)
  preps = randompreparations(N, ntrial)

  for n in 1:ntrial
    mgates = PastaQ.measurementgates(bases[n, :])
    ψ_in = productstate(N, preps[n, :])
    ψ_out = runcircuit(ψ_in, gates)

    Ψ_out = PastaQ.projectchannel(U, preps[n, :])
    @test PastaQ.array(ψ_out) ≈ PastaQ.array(Ψ_out)

    ψ_m = runcircuit(ψ_out, mgates)
    Ψ_m = runcircuit(Ψ_out, mgates)
    @test PastaQ.array(ψ_m) ≈ PastaQ.array(Ψ_m)
  end
end

@testset "project Choi MPO" begin
  N = 4
  ntrial = 100
  gates = randomcircuit(N; depth=4, layered=false)

  Λ = runcircuit(N, gates; process=true, noise=("depolarizing", (p=0.1,)))

  bases = randombases(N, ntrial)
  preps = PastaQ.randompreparations(N, ntrial)
  for n in 1:ntrial
    mgates = PastaQ.measurementgates(bases[n, :])
    ψ_in = productstate(N, preps[n, :])
    ρ_out = runcircuit(ψ_in, gates; noise=("depolarizing", (p=0.1,)))

    Λ_out = PastaQ.projectchannel(Λ, preps[n, :])
    @test PastaQ.array(ρ_out) ≈ PastaQ.array(Λ_out) atol = 1e-6

    ρ_m = runcircuit(ρ_out, mgates)
    Λ_m = runcircuit(Λ_out, mgates)
    @test PastaQ.array(ρ_m) ≈ PastaQ.array(Λ_m) atol = 1e-6
  end
end

@testset "project Choi ITensor" begin
  N = 4
  ntrial = 100
  gates = randomcircuit(N; depth=4, layered=false)

  @disable_warn_order begin
    Λ = runcircuit(
      N, gates; process=true, noise=("depolarizing", (p=0.1,)), full_representation=true
    )

    bases = randombases(N, ntrial)
    preps = PastaQ.randompreparations(N, ntrial)
    for n in 1:ntrial
      mgates = PastaQ.measurementgates(bases[n, :])
      ψ_in = productstate(N, preps[n, :])
      ρ_out = runcircuit(ψ_in, gates; noise=("depolarizing", (p=0.1,)))

      Λ_out = PastaQ.projectchannel(Λ, preps[n, :])
      @test PastaQ.array(ρ_out) ≈ PastaQ.array(Λ_out) atol = 1e-6

      ρ_m = runcircuit(ρ_out, mgates)
      Λ_m = runcircuit(Λ_out, mgates)
      @test PastaQ.array(ρ_m) ≈ PastaQ.array(Λ_m) atol = 1e-6
    end
  end
end

@testset "getsamples: states" begin
  N = 3
  circuit = randomcircuit(N; depth=3)

  # quantum states
  ψ = runcircuit(N, circuit)
  ρ = runcircuit(N, circuit; noise=("depolarizing", (p=0.1,)))

  nbases = 11
  bases = randombases(N, nbases)
  data = getsamples(ψ, bases)
  @test size(data) == (nbases, N)
  nshots = 3
  data = getsamples(ψ, bases, nshots)
  @test size(data) == (nbases * nshots, N)
  for b in 1:nbases
    for k in 1:nshots
      @test first.(data[(b - 1) * nshots + k, :]) == bases[b, :]
    end
  end

  data = getsamples(ρ, bases)
  @test size(data) == (nbases, N)
  nshots = 3
  data = getsamples(ρ, bases, nshots)
  @test size(data) == (nbases * nshots, N)
  for b in 1:nbases
    for k in 1:nshots
      @test first.(data[(b - 1) * nshots + k, :]) == bases[b, :]
    end
  end

  # ITensors quantum states
  ψ = runcircuit(N, circuit; full_representation=true)
  ρ = runcircuit(N, circuit; noise=("depolarizing", (p=0.1,)), full_representation=true)

  nbases = 11
  bases = randombases(N, nbases)
  data = getsamples(ψ, bases)
  @test size(data) == (nbases, N)
  nshots = 3
  data = getsamples(ψ, bases, nshots)
  @test size(data) == (nbases * nshots, N)
  for b in 1:nbases
    for k in 1:nshots
      @test first.(data[(b - 1) * nshots + k, :]) == bases[b, :]
    end
  end

  data = getsamples(ρ, bases)
  @test size(data) == (nbases, N)
  nshots = 3
  data = getsamples(ρ, bases, nshots)
  @test size(data) == (nbases * nshots, N)
  for b in 1:nbases
    for k in 1:nshots
      @test first.(data[(b - 1) * nshots + k, :]) == bases[b, :]
    end
  end
end

@testset "getsamples: channels" begin
  N = 3
  circuit = randomcircuit(N; depth=3)

  # quantum processes
  U = runcircuit(N, circuit; process=true)
  Λ = runcircuit(N, circuit; noise=("depolarizing", (p=0.1,)), process=true)

  npreps = 5
  nbases = 7
  preps = randompreparations(N, npreps)
  bases = randombases(N, nbases)

  data = getsamples(U, preps, bases)
  @test size(data) == (npreps * nbases, N)
  nshots = 3
  data = getsamples(U, preps, bases, nshots)
  @test size(data) == (npreps * nbases * nshots, N)

  for p in 1:npreps
    for b in 1:nbases
      for k in 1:nshots
        pdata = first.(data[(p - 1) * nbases * nshots + (b - 1) * nshots + k, :])
        bdata = first.(last.(data[(p - 1) * nbases * nshots + (b - 1) * nshots + k, :]))
        @test pdata == preps[p, :]
        @test bdata == bases[b, :]
      end
    end
  end

  npreps = 5
  nbases = 5
  preps = randompreparations(N, npreps)
  bases = randombases(N, nbases)

  data = getsamples(U, preps .=> bases)
  @test size(data) == (npreps, N)

  data = getsamples(Λ, preps, bases)
  @test size(data) == (npreps * nbases, N)
  nshots = 3
  data = getsamples(Λ, preps, bases, nshots)
  @test size(data) == (npreps * nbases * nshots, N)

  for p in 1:npreps
    for b in 1:nbases
      for k in 1:nshots
        pdata = first.(data[(p - 1) * nbases * nshots + (b - 1) * nshots + k, :])
        bdata = first.(last.(data[(p - 1) * nbases * nshots + (b - 1) * nshots + k, :]))
        @test pdata == preps[p, :]
        @test bdata == bases[b, :]
      end
    end
  end

  npreps = 5
  nbases = 5
  preps = randompreparations(N, npreps)
  bases = randombases(N, nbases)

  data = getsamples(Λ, preps .=> bases)
  @test size(data) == (npreps, N)

  # full representation
  U = runcircuit(N, circuit; process=true, full_representation=true)
  Λ = runcircuit(
    N, circuit; noise=("depolarizing", (p=0.1,)), process=true, full_representation=true
  )

  npreps = 5
  nbases = 7
  preps = randompreparations(N, npreps)
  bases = randombases(N, nbases)

  data = getsamples(U, preps, bases)
  @test size(data) == (npreps * nbases, N)
  nshots = 3
  data = getsamples(U, preps, bases, nshots)
  @test size(data) == (npreps * nbases * nshots, N)

  for p in 1:npreps
    for b in 1:nbases
      for k in 1:nshots
        pdata = first.(data[(p - 1) * nbases * nshots + (b - 1) * nshots + k, :])
        bdata = first.(last.(data[(p - 1) * nbases * nshots + (b - 1) * nshots + k, :]))
        @test pdata == preps[p, :]
        @test bdata == bases[b, :]
      end
    end
  end

  data = getsamples(Λ, preps, bases)
  @test size(data) == (npreps * nbases, N)
  nshots = 3
  data = getsamples(Λ, preps, bases, nshots)
  @test size(data) == (npreps * nbases * nshots, N)

  for p in 1:npreps
    for b in 1:nbases
      for k in 1:nshots
        pdata = first.(data[(p - 1) * nbases * nshots + (b - 1) * nshots + k, :])
        bdata = first.(last.(data[(p - 1) * nbases * nshots + (b - 1) * nshots + k, :]))
        @test pdata == preps[p, :]
        @test bdata == bases[b, :]
      end
    end
  end
end

In [ ]:
#Testing process tomography

In [ ]:
using PastaQ
using ITensors
using Test
using LinearAlgebra
using Random

""" HELPER FUNCTIONS """
function numgradslogZ(L::LPDO; accuracy=1e-8)
  M = L.X
  N = length(M)
  grad_r = []
  grad_i = []
  for j in 1:N
    push!(grad_r, zeros(ComplexF64, size(M[j])))
    push!(grad_i, zeros(ComplexF64, size(M[j])))
  end

  epsilon = zeros(ComplexF64, size(M[1]))
  # Site 1
  for i in 1:length(epsilon)
    epsilon[i] = accuracy
    eps = ITensor(epsilon, inds(M[1]))
    M[1] += eps
    loss_p = 2.0 * lognorm(M)
    M[1] -= eps
    loss_m = 2.0 * lognorm(M)
    grad_r[1][i] = (loss_p - loss_m) / (accuracy)

    epsilon[i] = im * accuracy
    eps = ITensor(epsilon, inds(M[1]))
    M[1] += eps
    loss_p = 2.0 * lognorm(M)
    M[1] -= eps
    loss_m = 2.0 * lognorm(M)
    grad_i[1][i] = (loss_p - loss_m) / (im * accuracy)

    epsilon[i] = 0.0
  end

  for j in 2:(N - 1)
    epsilon = zeros(ComplexF64, size(M[j]))
    for i in 1:length(epsilon)
      epsilon[i] = accuracy
      eps = ITensor(epsilon, inds(M[j]))
      M[j] += eps
      loss_p = 2.0 * lognorm(M)
      M[j] -= eps
      loss_m = 2.0 * lognorm(M)
      grad_r[j][i] = (loss_p - loss_m) / (accuracy)

      epsilon[i] = im * accuracy
      eps = ITensor(epsilon, inds(M[j]))
      M[j] += eps
      loss_p = 2.0 * lognorm(M)
      M[j] -= eps
      loss_m = 2.0 * lognorm(M)
      grad_i[j][i] = (loss_p - loss_m) / (im * accuracy)

      epsilon[i] = 0.0
    end
  end
  # Site N
  epsilon = zeros(ComplexF64, size(M[N]))
  for i in 1:length(epsilon)
    epsilon[i] = accuracy
    eps = ITensor(epsilon, inds(M[N]))
    M[N] += eps
    loss_p = 2.0 * lognorm(M)
    M[N] -= eps
    loss_m = 2.0 * lognorm(M)
    grad_r[N][i] = (loss_p - loss_m) / (accuracy)

    epsilon[i] = im * accuracy
    eps = ITensor(epsilon, inds(M[N]))
    M[N] += eps
    loss_p = 2.0 * lognorm(M)
    M[N] -= eps
    loss_m = 2.0 * lognorm(M)
    grad_i[N][i] = (loss_p - loss_m) / (im * accuracy)

    epsilon[i] = 0.0
  end

  return grad_r - grad_i
end

#numgradslogZ(M::MPS; kwargs...) = numgradslogZ(LPDO(M); kwargs...)

function numgradsnll(L::LPDO, data::Matrix{Pair{String,Pair{String,Int}}}, accuracy=1e-8)
  M = L.X
  N = length(M)
  grad_r = []
  grad_i = []
  for j in 1:N
    push!(grad_r, zeros(ComplexF64, size(M[j])))
    push!(grad_i, zeros(ComplexF64, size(M[j])))
  end

  epsilon = zeros(ComplexF64, size(M[1]))
  # Site 1
  for i in 1:length(epsilon)
    epsilon[i] = accuracy
    eps = ITensor(epsilon, inds(M[1]))
    M[1] += eps
    loss_p = PastaQ.nll(L, data)
    M[1] -= eps
    loss_m = PastaQ.nll(L, data)
    grad_r[1][i] = (loss_p - loss_m) / (accuracy)

    epsilon[i] = im * accuracy
    eps = ITensor(epsilon, inds(M[1]))
    M[1] += eps
    loss_p = PastaQ.nll(L, data)
    M[1] -= eps
    loss_m = PastaQ.nll(L, data)
    grad_i[1][i] = (loss_p - loss_m) / (im * accuracy)

    epsilon[i] = 0.0
  end

  for j in 2:(N - 1)
    epsilon = zeros(ComplexF64, size(M[j]))
    for i in 1:length(epsilon)
      epsilon[i] = accuracy
      eps = ITensor(epsilon, inds(M[j]))
      M[j] += eps
      loss_p = PastaQ.nll(L, data)
      M[j] -= eps
      loss_m = PastaQ.nll(L, data)
      grad_r[j][i] = (loss_p - loss_m) / (accuracy)

      epsilon[i] = im * accuracy
      eps = ITensor(epsilon, inds(M[j]))
      M[j] += eps
      loss_p = PastaQ.nll(L, data)
      M[j] -= eps
      loss_m = PastaQ.nll(L, data)
      grad_i[j][i] = (loss_p - loss_m) / (im * accuracy)

      epsilon[i] = 0.0
    end
  end

  # Site N
  epsilon = zeros(ComplexF64, size(M[N]))
  for i in 1:length(epsilon)
    epsilon[i] = accuracy
    eps = ITensor(epsilon, inds(M[N]))
    M[N] += eps
    loss_p = PastaQ.nll(L, data)
    M[N] -= eps
    loss_m = PastaQ.nll(L, data)
    grad_r[N][i] = (loss_p - loss_m) / (accuracy)

    epsilon[i] = im * accuracy
    eps = ITensor(epsilon, inds(M[N]))
    M[N] += eps
    loss_p = PastaQ.nll(L, data)
    M[N] -= eps
    loss_m = PastaQ.nll(L, data)
    grad_i[N][i] = (loss_p - loss_m) / (im * accuracy)

    epsilon[i] = 0.0
  end

  return grad_r - grad_i
end

function numgradsTP(L::LPDO; accuracy=1e-8)
  M = L.X
  N = length(M)
  grad_r = []
  grad_i = []

  for j in 1:N
    push!(grad_r, zeros(ComplexF64, size(M[j])))
    push!(grad_i, zeros(ComplexF64, size(M[j])))
  end

  epsilon = zeros(ComplexF64, size(M[1]))
  # Site 1
  for i in 1:length(epsilon)
    epsilon[i] = accuracy
    eps = ITensor(epsilon, inds(M[1]))
    M[1] += eps
    loss_p = PastaQ.TP(L)
    M[1] -= eps
    loss_m = PastaQ.TP(L)
    grad_r[1][i] = (loss_p - loss_m) / (accuracy)

    epsilon[i] = im * accuracy
    eps = ITensor(epsilon, inds(M[1]))
    M[1] += eps
    loss_p = PastaQ.TP(L)
    M[1] -= eps
    loss_m = PastaQ.TP(L)
    grad_i[1][i] = (loss_p - loss_m) / (im * accuracy)

    epsilon[i] = 0.0
  end

  for j in 2:(N - 1)
    epsilon = zeros(ComplexF64, size(M[j]))
    for i in 1:length(epsilon)
      epsilon[i] = accuracy
      eps = ITensor(epsilon, inds(M[j]))
      M[j] += eps
      loss_p = PastaQ.TP(L)
      M[j] -= eps
      loss_m = PastaQ.TP(L)
      grad_r[j][i] = (loss_p - loss_m) / (accuracy)

      epsilon[i] = im * accuracy
      eps = ITensor(epsilon, inds(M[j]))
      M[j] += eps
      loss_p = PastaQ.TP(L)
      M[j] -= eps
      loss_m = PastaQ.TP(L)
      grad_i[j][i] = (loss_p - loss_m) / (im * accuracy)

      epsilon[i] = 0.0
    endU
  end

  # Site N
  epsilon = zeros(ComplexF64, size(M[N]))
  for i in 1:length(epsilon)
    epsilon[i] = accuracy
    eps = ITensor(epsilon, inds(M[N]))
    M[N] += eps
    loss_p = PastaQ.TP(L)
    M[N] -= eps
    loss_m = PastaQ.TP(L)
    grad_r[N][i] = (loss_p - loss_m) / (accuracy)
    epsilon[i] = im * accuracy
    eps = ITensor(epsilon, inds(M[N]))
    M[N] += eps
    loss_p = PastaQ.TP(L)
    M[N] -= eps
    loss_m = PastaQ.TP(L)
    grad_i[N][i] = (loss_p - loss_m) / (im * accuracy)

    epsilon[i] = 0.0
  end

  return grad_r - grad_i
end

@testset "mpo-qpt: normalization" begin
  N = 5
  χ = 4
  U = randomprocess(N; χ=χ)
  Λ = LPDO(PastaQ.unitary_mpo_to_choi_mps(U))
  @test length(Λ) == N
  logZ = 2 * lognorm(Λ.X)
  sqrt_localZ = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localZ, localnorm=2)
  @test logZ ≈ N * log(2) + 2.0 * sum(log.(sqrt_localZ))
  @test abs2(norm(Λ.X)) ≈ 2^N
end

@testset "mpo-qpt: grad logZ" begin
  N = 5
  χ = 4

  Random.seed!(1234)
  U = randomprocess(N; χ=χ)
  Λ = LPDO(PastaQ.unitary_mpo_to_choi_mps(U))
  num_grad = numgradslogZ(Λ)

  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)
  @test norm(Λ.X)^2 ≈ 2^N
  alg_grad, _ = PastaQ.gradlogZ(Λ; sqrt_localnorms=sqrt_localnorms)

  alg_gradient = permutedims(ITensors.array(alg_grad[1]), [1, 3, 2])
  @test alg_gradient ≈ num_grad[1] rtol = 1e-3
  for j in 2:(N - 1)
    alg_gradient = permutedims(ITensors.array(alg_grad[j]), [2, 1, 3, 4])
    @test alg_gradient ≈ num_grad[j] rtol = 1e-3
  end
  alg_gradient = permutedims(ITensors.array(alg_grad[N]), [2, 1, 3])
  @test alg_gradient ≈ num_grad[N] rtol = 1e-3
end

@testset "mpo-qpt: grad nll" begin
  N = 4
  χ = 2
  nsamples = 10
  Random.seed!(1234)
  data_in = randompreparations(N, nsamples)
  data_out = PastaQ.convertdatapoints(randompreparations(N, nsamples))
  data = data_in .=> data_out

  U = randomprocess(N; χ=χ)
  Λ = LPDO(PastaQ.unitary_mpo_to_choi_mps(U))
  num_grad = numgradsnll(Λ, data)
  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)

  alg_grad, _ = PastaQ.gradnll(Λ, data; sqrt_localnorms=sqrt_localnorms)
  for j in 1:N
    @test ITensors.array(alg_grad[j]) ≈ num_grad[j] rtol = 1e-3
  end
end

@testset "mpo-qpt: grad TP" begin
  N = 3
  χ = 3
  Random.seed!(1234)

  Random.seed!(1234)
  U = randomprocess(N; χ=χ)
  Λ = LPDO(PastaQ.unitary_mpo_to_choi_mps(U))

  num_grad = numgradsTP(Λ; accuracy=1e-8)
  Γ_test = PastaQ.TP(Λ)
  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)

  alg_grad_logZ, logZ = PastaQ.gradlogZ(Λ; sqrt_localnorms=sqrt_localnorms)

  alg_grad, Γ = PastaQ.gradTP(Λ, alg_grad_logZ, logZ; sqrt_localnorms=sqrt_localnorms)

  @test Γ ≈ Γ_test
  alg_gradient = permutedims(ITensors.array(alg_grad[1]), [1, 3, 2])
  @test alg_gradient ≈ num_grad[1] rtol = 1e-5
  for j in 2:(N - 1)
    alg_gradient = permutedims(ITensors.array(alg_grad[j]), [1, 3, 2, 4])
    @test alg_gradient ≈ num_grad[j] rtol = 1e-5
  end
  alg_gradient = permutedims(ITensors.array(alg_grad[N]), [1, 3, 2])
  @test alg_gradient ≈ num_grad[N] rtol = 1e-5
end

@testset "mpo-qpt: full gradients" begin
  N = 3
  χ = 4
  nsamples = 10
  trace_preserving_regularizer = 0.1
  Random.seed!(1234)
  data_in = randompreparations(N, nsamples)
  data_out = PastaQ.convertdatapoints(randompreparations(N, nsamples))
  data = data_in .=> data_out

  U = randomprocess(N; χ=χ)
  Λ = LPDO(PastaQ.unitary_mpo_to_choi_mps(U))
  TP_distance = PastaQ.TP(Λ)
  logZ = log(tr(Λ))
  NLL = PastaQ.nll(Λ, data)
  num_gradZ = numgradslogZ(Λ)
  num_gradNLL = numgradsnll(Λ, data)
  num_gradTP = numgradsTP(Λ; accuracy=1e-5)

  num_grads = num_gradZ + num_gradNLL + trace_preserving_regularizer * num_gradTP

  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)

  ex_loss = PastaQ.nll(Λ, data) + 2 * lognorm(Λ.X)
  
  display(Λ)
  print(" ")
    
  display(data)
  alg_grads, loss = PastaQ.gradients(
    Λ,
    data;
    sqrt_localnorms=sqrt_localnorms,
    trace_preserving_regularizer=trace_preserving_regularizer,
  )
  @test ex_loss ≈ loss
  for j in 1:N
    @test ITensors.array(alg_grads[j]) ≈ num_grads[j] rtol = 1e-3
  end
end

""" CHOI TESTS """

@testset "lpdo-qpt: normalization" begin
  N = 10
  χ = 4
  ξ = 3
  Λ = randomprocess(N; mixed=true, ξ=ξ, χ=χ)

  @test length(Λ) == N
  logZ = logtr(Λ)
  localZ = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=localZ, localnorm=2)
  @test logZ ≈ N * log(2) + 2.0 * sum(log.(localZ))
  @test tr(Λ) ≈ 2^N
end

@testset "lpdo-qst: grad logZ" begin
  N = 5
  χ = 4
  ξ = 3

  Λ = randomprocess(N; mixed=true, χ=χ, ξ=ξ)
  num_grad = numgradslogZ(Λ)
  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)
  @test tr(Λ) ≈ 2^N
  alg_grad, _ = PastaQ.gradlogZ(Λ; sqrt_localnorms=sqrt_localnorms)

  alg_gradient = permutedims(ITensors.array(alg_grad[1]), [1, 2, 4, 3])
  for j in 2:(N - 1)
    alg_gradient = permutedims(ITensors.array(alg_grad[j]), [2, 3, 1, 4, 5])
    @test alg_gradient ≈ num_grad[j] rtol = 1e-3
  end
  alg_gradient = permutedims(ITensors.array(alg_grad[N]), [2, 3, 1, 4])
  @test alg_gradient ≈ num_grad[N] rtol = 1e-3
end

@testset "lpdo-qst: grad nll" begin
  N = 5
  χ = 4
  ξ = 3

  nsamples = 10
  Random.seed!(1234)
  data_in = randompreparations(N, nsamples)
  data_out = PastaQ.convertdatapoints(randompreparations(N, nsamples))
  data = data_in .=> data_out

  Λ = randomprocess(N; mixed=true, χ=χ, ξ=ξ)
  num_grad = numgradsnll(Λ, data)
  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)
  alg_grad, loss = PastaQ.gradnll(Λ, data; sqrt_localnorms=sqrt_localnorms)
  @test loss ≈ PastaQ.nll(Λ, data)

  alg_gradient = permutedims(ITensors.array(alg_grad[1]), [3, 4, 1, 2])
  @test alg_gradient ≈ num_grad[1] rtol = 1e-3
  for j in 2:(N - 1)
    alg_gradient = permutedims(ITensors.array(alg_grad[j]), [4, 5, 2, 3, 1])
    @test alg_gradient ≈ num_grad[j] rtol = 1e-3
  end
  alg_gradient = permutedims(ITensors.array(alg_grad[N]), [3, 4, 1, 2])
  @test alg_gradient ≈ num_grad[N] rtol = 1e-3
end

@testset "lpdo-qpt: grad TP" begin
  N = 3
  χ = 4
  ξ = 3
  Λ = randomprocess(N; mixed=true, χ=χ, ξ=ξ)

  num_grad = numgradsTP(Λ; accuracy=1e-8)
  Γ_test = PastaQ.TP(Λ)
  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)
  Γ_test = PastaQ.TP(Λ)

  alg_grad_logZ, logZ = PastaQ.gradlogZ(Λ; sqrt_localnorms=sqrt_localnorms)

  alg_grad, Γ = PastaQ.gradTP(Λ, alg_grad_logZ, logZ; sqrt_localnorms=sqrt_localnorms)

  @test Γ ≈ Γ_test
  alg_gradient = permutedims(ITensors.array(alg_grad[1]), [3, 1, 4, 2])
  @test alg_gradient ≈ num_grad[1] rtol = 1e-3
  for j in 2:(N - 1)
    alg_gradient = permutedims(ITensors.array(alg_grad[j]), [3, 1, 4, 2, 5])
    @test alg_gradient ≈ num_grad[j] rtol = 1e-3
  end
  alg_gradient = permutedims(ITensors.array(alg_grad[N]), [3, 1, 4, 2])
  @test alg_gradient ≈ num_grad[N] rtol = 1e-3
end

@testset "lpdo-qpt: full gradients" begin
  N = 3
  χ = 3
  ξ = 2

  trace_preserving_regularizer = 0.1
  nsamples = 10
  Random.seed!(1234)
  data_in = randompreparations(N, nsamples)
  data_out = PastaQ.convertdatapoints(randompreparations(N, nsamples))
  data = data_in .=> data_out

  Λ = randomprocess(N; mixed=true, χ=χ, ξ=ξ)
  num_grad = numgradsnll(Λ, data)
  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)
  alg_grad, loss = PastaQ.gradnll(Λ, data; sqrt_localnorms=sqrt_localnorms)

  TP_distance = PastaQ.TP(Λ)
  logZ = log(tr(Λ))
  NLL = PastaQ.nll(Λ, data)
  num_gradZ = numgradslogZ(Λ)
  num_gradNLL = numgradsnll(Λ, data)
  num_gradTP = numgradsTP(Λ; accuracy=1e-5)

  num_grads = num_gradZ + num_gradNLL + trace_preserving_regularizer * num_gradTP

  sqrt_localnorms = []
  PastaQ.normalize!(Λ; (sqrt_localnorms!)=sqrt_localnorms, localnorm=2)

  ex_loss = PastaQ.nll(Λ, data) + 2 * lognorm(Λ.X)
  alg_grads, loss = PastaQ.gradients(
    Λ,
    data;
    sqrt_localnorms=sqrt_localnorms,
    trace_preserving_regularizer=trace_preserving_regularizer,
  )
  @test ex_loss ≈ loss
  for j in 1:N
    @test ITensors.array(alg_grads[j]) ≈ num_grads[j] rtol = 1e-3
  end
end

In [ ]:
using PastaQ
using Random
using ITensors
using Observers
using Optimisers: Optimisers

Random.seed!(1234)

# 1. Quantum process tomography of a unitary circuit

# Make the random circuit
N = 3
depth = 4
nshots = 100
circuit = randomcircuit(N,depth=depth)
ρ = runcircuit(circuit)

#print(circuit)
# Generate samples
#print(bases)
#data,Û= getsamples(ρ, bases,nshots)
data = getsamples(ρ, nshots)


#writesamples(data, U, "data/qpt_circuit.h5")

# Load target state and measurements. Each samples is built out
# of an input state (`first.(data)`) to the quantum channel, and the
# measurement output (`last.(data)`) after a local basis rotation.
#data, Û = readsamples("data/qpt_circuit.h5")

# Split data into a train and test dataset
train_data, test_data = split_dataset(data; train_ratio=0.9)

display(train_data)

# Set parameters
N = length(Û)     # Number of qubits
χ = maxlinkdim(Û) # Bond dimension of variational MPS

# Initialize the unitary MPO
U0 = randomprocess(Û; χ=χ)

opt = Optimisers.Descent(0.01)

F(U::MPO; kwargs...) = fidelity(U, Û; process=true)
obs = Observer(["F" => F])

# Initialize stochastic gradient descent optimizer
@show maxlinkdim(U0)

# Run process tomography
println("Run process tomography to learn noiseless circuit U")
U = tomography(
  train_data,
  U0;
  test_data=test_data,
  optimizer=opt,
  batchsize=500,
  epochs=5,
  (observer!)=obs,
  print_metrics=["F"],
)

@show maxlinkdim(U)
println()

# Noisy circuit
# Generate samples
data, Λ = getsamples(
  circuit,
  nshots;
  local_basis=["X", "Y", "Z"],
  process=true,
  noise=("amplitude_damping", (γ=0.01,)),
)
writesamples(data, Λ, "data/qpt_circuit_noisy.h5")

# Load data and target Choi matrix
data, Φ = readsamples("data/qpt_circuit_noisy.h5")

# Split data into a train and test dataset
train_data, test_data = split_dataset(data; train_ratio=0.9)

# Set up
N = length(Φ)
χ = 8
ξ = 2

# Initialize the Choi LPDO
Λ0 = randomprocess(Φ; mixed=true, χ=χ, ξ=ξ)

# Initialize stochastic gradient descent optimizer
opt = Optimisers.ADAM()

F(Λ::LPDO; kwargs...) = fidelity(Λ, Φ; process=true)
obs = Observer(["F" => F])

# Run process tomography
println("Run process tomography to learn noisy process Λ")
@disable_warn_order begin
  Λ = tomography(
    train_data,
    Λ0;
    test_data=test_data,
    optimizer=opt,
    batchsize=500,
    epochs=5,
    (observer!)=obs,
    print_metrics=["F"],
  )
end
@show maxlinkdim(Λ.X)
println()